In [1]:
import sys
sys.path.append('/Users/pradap/Documents/Research/Python-Package/enrique/')

In [5]:
%load_ext autotime

In [2]:
import magellan as mg

DEBUG:Cloud:Log file (/Users/pradap/.picloud/cloud.log) opened


In [3]:
mg.init_jvm()

True

In [4]:
path_A = 'imdbClean.csv'
path_B = 'tmdClean.csv'

# read table A; table A has 'ID as the key attribute
A = mg.read_csv(path_A, key='ID')
B = mg.read_csv(path_B, key='ID')

In [6]:
# Read in I feature vector: director
I = mg.read_csv('train_set_director.csv', ltable=A, rtable=B)

time: 4.86 ms


In [7]:
I.head()

,_id,ltable.ID,rtable.ID,director_director_jac_qgm_3_qgm_3,director_director_cos_dlm_dc0_dlm_dc0,director_director_jac_dlm_dc0_dlm_dc0,director_director_mel,director_director_lev,director_director_nmw,director_director_sw,director_director_swg,gold_label
0,94,488,1108,0.900000,1.0,1.000000,1.000000,0.916667,1.000000,1.000000,1.000000,1
1,350,2594,2403,0.150000,0.5,0.333333,0.416667,0.357143,0.642857,0.416667,0.416667,0
2,42,223,316,0.909091,1.0,1.000000,1.000000,0.923077,1.000000,1.000000,1.000000,1
3,387,2940,2108,0.941176,1.0,1.000000,1.000000,0.947368,1.000000,1.000000,1.000000,1
4,91,479,510,0.916667,1.0,1.000000,1.000000,0.928571,1.000000,1.000000,1.000000,0


time: 23.4 ms


In [8]:
# debug lg
lg = mg.LogRegMatcher()

time: 1.26 ms


In [9]:
# spliting data into U and V
dic = mg.train_test_split(I, train_proportion=0.7)
U = dic['train']
V = dic['test']

time: 7.64 ms


In [10]:
# train lg using U
lg.fit(table = U, exclude_attrs = ['_id', 'ltable.ID', 'rtable.ID', 'gold_label'],
                             target_attr = 'gold_label')

time: 10.5 ms


In [11]:
# predict on V
P5 = lg.predict(table = V, exclude_attrs = ['_id', 'ltable.ID', 'rtable.ID', 'gold_label'],
                             target_attr = 'predicted_label', append = True, inplace= False)

time: 10.5 ms


In [13]:
P5.head()

,_id,ltable.ID,rtable.ID,director_director_jac_qgm_3_qgm_3,director_director_cos_dlm_dc0_dlm_dc0,director_director_jac_dlm_dc0_dlm_dc0,director_director_mel,director_director_lev,director_director_nmw,director_director_sw,director_director_swg,gold_label,predicted_label
0,330,2322,1852,0.075000,0.353553,0.20,0.371429,0.242424,0.546875,0.357143,0.400000,0,0
1,101,533,1531,0.160000,0.408248,0.25,0.461538,0.421053,0.578947,0.461538,0.492308,0,0
2,69,357,159,0.916667,1.000000,1.00,1.000000,0.928571,1.000000,1.000000,1.000000,1,1
3,307,2159,1544,0.888889,1.000000,1.00,1.000000,0.909091,1.000000,1.000000,1.000000,1,1
4,310,2171,1421,0.875000,1.000000,1.00,1.000000,0.900000,1.000000,1.000000,1.000000,1,1


time: 22.7 ms


In [14]:
# evaluation on predictions
eval_summary5 = mg.eval_matches(P5, 'gold_label', 'predicted_label')

time: 11.3 ms


In [15]:
mg.print_eval_summary(eval_summary5)

Precision : 93.55% (58/62)
Recall : 98.31% (58/59)
F1 : 95.87%
False positives : 4 (out of 62 positive predictions)
False negatives : 1 (out of 22 negative predictions)
time: 1.06 ms


In [16]:
for p in eval_summary5:
    print p + ': ' + str(eval_summary5[p])

prec_numerator: 58.0
prec_denominator: 62.0
precision: 0.935483870968
recall_numerator: 58.0
recall_denominator: 59.0
recall: 0.983050847458
f1: 0.95867768595
pred_pos_num: 62.0
false_pos_num: 4.0
false_pos_ls: [(106.0, 224.0), (305.0, 958.0), (106.0, 117.0), (479.0, 510.0)]
pred_neg_num: 22.0
false_neg_num: 1.0
false_neg_ls: [(171.0, 63.0)]
time: 1.37 ms


In [13]:
# write vector table U and V out
U.to_csv('vector_table_U.csv')
V.to_csv('vector_table_V.csv')

True

In [22]:
A[A.ID==2322]

,ID,title,year,director,writer,actor
2300,2322,A Night at the Roxbury,1998,John Fortenberry; Amy Heckerling,Steve Koren; Will Ferrell,Chris Kattan; Will Ferrell; Raquel Gardner


time: 10 ms


In [17]:
P5.head()

,_id,ltable.ID,rtable.ID,director_director_jac_qgm_3_qgm_3,director_director_cos_dlm_dc0_dlm_dc0,director_director_jac_dlm_dc0_dlm_dc0,director_director_mel,director_director_lev,director_director_nmw,director_director_sw,director_director_swg,gold_label,predicted_label
0,330,2322,1852,0.075000,0.353553,0.20,0.371429,0.242424,0.546875,0.357143,0.400000,0,0
1,101,533,1531,0.160000,0.408248,0.25,0.461538,0.421053,0.578947,0.461538,0.492308,0,0
2,69,357,159,0.916667,1.000000,1.00,1.000000,0.928571,1.000000,1.000000,1.000000,1,1
3,307,2159,1544,0.888889,1.000000,1.00,1.000000,0.909091,1.000000,1.000000,1.000000,1,1
4,310,2171,1421,0.875000,1.000000,1.00,1.000000,0.900000,1.000000,1.000000,1.000000,1,1


time: 16.9 ms


In [18]:
feat_table = mg.get_features_for_matching(A, B)

time: 83.8 ms


In [19]:
feat_table.head()

,feature_name,left_attribute,right_attribute,left_attr_tokenizer,right_attr_tokenizer,simfunction,function,function_source
0,ID_ID_exm,ID,ID,None,None,exact_match,<function ID_ID_exm at 0x1202a68c0>,from magellan.feature.simfunctions import *\nf...
1,ID_ID_anm,ID,ID,None,None,abs_norm,<function ID_ID_anm at 0x1202a6758>,from magellan.feature.simfunctions import *\nf...
2,ID_ID_lev,ID,ID,None,None,lev,<function ID_ID_lev at 0x1202a6230>,from magellan.feature.simfunctions import *\nf...
3,title_title_jac_qgm_3_qgm_3,title,title,qgm_3,qgm_3,jaccard,<function title_title_jac_qgm_3_qgm_3 at 0x120...,from magellan.feature.simfunctions import *\nf...
4,title_title_cos_dlm_dc0_dlm_dc0,title,title,dlm_dc0,dlm_dc0,cosine,<function title_title_cos_dlm_dc0_dlm_dc0 at 0...,from magellan.feature.simfunctions import *\nf...


time: 14.2 ms


In [23]:
# Add trigger - target false positives: use title related feature
neg_trigger = mg.MatchTrigger()
neg_trigger.add_cond_rule('title_title_jac_qgm_3_qgm_3(ltuple, rtuple) < 0.4', feat_table)
neg_trigger.add_cond_status(True)
neg_trigger.add_action(0)

True

time: 3.21 ms


In [27]:
neg_trigger.rule_source['_rule_0']

'def _rule_0(ltuple, rtuple):\n    return title_title_jac_qgm_3_qgm_3(ltuple, rtuple) < 0.4'

time: 2.02 ms


In [31]:
# Pradap
P5['ltable.ID'] = P5['ltable.ID'].astype('int')
P5['rtable.ID'] = P5['rtable.ID'].astype('int')


time: 2.12 ms


In [32]:
# Apply trigger and then evaluate
# Apply trigger
Q = neg_trigger.execute(P5, 'predicted_label', inplace=False)

time: 67.3 ms


In [20]:
help(mg.MatchTrigger())

Help on MatchTrigger in module magellan.trigger.matchtrigger object:

class MatchTrigger(__builtin__.object)
 |  Class to patch output from Matchers.
 |  Note:
 |  This class is similar to BooleanRuleMatcher except that this class switches predictions.
 |  
 |  Methods defined here:
 |  
 |  __init__(self)
 |  
 |  add_action(self, value)
 |      Add action if the condition evaluates to condition status (see add_cond_rule, add_cond_status)
 |      
 |      Parameters
 |      ----------
 |      value : int (0/1). Value to be set if the condition evaluates to condition status
 |      
 |      Returns
 |      -------
 |      status : boolean, returns True if the command was executed successfully.
 |  
 |  add_cond_rule(self, conjunct_list, feature_table)
 |      Add rule to match trigger
 |      
 |      Parameters
 |      ----------
 |      conjunct_list : List of strings.
 |       List of predicates as in ['title_title_lev(ltuple, rtuple) > 0.8', 'name_name_mel(ltuple, rtuple)' > 0.6]
 